<a href="https://colab.research.google.com/github/Linaqruf/Scraper/blob/main/Proseka_Wiki_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import requests
import re
import json
import os
import threading
from bs4 import BeautifulSoup
from pprint import pprint

base_url = 'https://projectsekai.fandom.com/wiki/Cards_'
step = 50
urls = []

for i in range(1, 650, step):
    start_range = i
    end_range = min(i + step - 1, 650)
    url = f"{base_url}{start_range}-{end_range}"
    urls.append(url)

def download(directory, url):
    if not os.path.exists(directory + url.split('/')[-1]):
        with open(directory + url.split('/')[-1], 'wb') as f:
            f.write(requests.get(url).content)
    else:
        return 0

def get_rarity(img_url):
    if 'Ribbon' in img_url:
        return 'ribbon'
    elif 'Star_4_trained' in img_url:
        return '4'
    elif 'Star_3_trained' in img_url:
        return '3'
    elif 'Star_2_untrained' in img_url:
        return '2'
    elif 'Star_untrained' in img_url:
        return '1'
    else:
        return ''

final = {}

for url_index, url in enumerate(urls):
    print(f"Processing URL {url_index + 1}/{len(urls)}: {url}")
    for i in BeautifulSoup(requests.get(url).content, 'lxml').find('table').findAll('tr')[1:]:
        td_elements = i.findAll('td')
        data = []
        for j in td_elements:
            if '<br/>' in str(j):
                names = []
                for link in j.findAll("a"):
                    names.append(link.text.strip())
                data.append(": ".join(names))
            else:
                data.append(j.text.strip())
        rarity = get_rarity(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[0].find('img')))[0])
        id = int(data[1])
        name = data[2]
        untrained_url = None
        trained_url = None
        if td_elements[3].find('img'):
            untrained_url = '/'.join(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[3].find('img')))[0].split('/')[:8])
        if td_elements[4].find('img'):
            trained_url = '/'.join(re.findall(r'src="(https:\/\/.*?)"', str(td_elements[4].find('img')))[0].split('/')[:8])
        card = {"rarity": rarity, "id": id, "name": name, "untrained_url": untrained_url, "trained_url": trained_url}

        if rarity in final:
            final[rarity].append(card)
        else:
            final[rarity] = [card]

        for card in final[rarity]:
            rarity_folder = f"proseka/images/untrained_card/rarity-{card['rarity']}"
            os.makedirs(rarity_folder, exist_ok=True)
            if card['untrained_url']:
                threading.Thread(target=download, args=(rarity_folder + '/', card['untrained_url'])).start()

            rarity_folder = f"proseka/images/trained_card/rarity-{card['rarity']}"
            os.makedirs(rarity_folder, exist_ok=True)
            if card['trained_url']:
                threading.Thread(target=download, args=(rarity_folder + '/', card['trained_url'])).start()

metadata_folder = f"proseka/metadata"
os.makedirs(metadata_folder, exist_ok=True)

for rarity, cards in final.items():
    with open(f'{metadata_folder}/rarity-{rarity}.json', 'w', encoding='utf-8') as f:
        json.dump(cards, f, indent=4, ensure_ascii=False)



Processing URL 1/13: https://projectsekai.fandom.com/wiki/Cards_1-50
Processing URL 2/13: https://projectsekai.fandom.com/wiki/Cards_51-100
Processing URL 3/13: https://projectsekai.fandom.com/wiki/Cards_101-150
Processing URL 4/13: https://projectsekai.fandom.com/wiki/Cards_151-200
Processing URL 5/13: https://projectsekai.fandom.com/wiki/Cards_201-250
Processing URL 6/13: https://projectsekai.fandom.com/wiki/Cards_251-300
Processing URL 7/13: https://projectsekai.fandom.com/wiki/Cards_301-350
Processing URL 8/13: https://projectsekai.fandom.com/wiki/Cards_351-400
Processing URL 9/13: https://projectsekai.fandom.com/wiki/Cards_401-450
Processing URL 10/13: https://projectsekai.fandom.com/wiki/Cards_451-500
Processing URL 11/13: https://projectsekai.fandom.com/wiki/Cards_501-550
Processing URL 12/13: https://projectsekai.fandom.com/wiki/Cards_551-600
Processing URL 13/13: https://projectsekai.fandom.com/wiki/Cards_601-650
